In [102]:
from nltk.tokenize import sent_tokenize,word_tokenize
from collections import Counter
import random
import numpy as np
from sklearn.model_selection import train_test_split

In [103]:
f=open("Project Gutenberg's The Complete Works of Jane Austen, by Jane Austen.txt","r")

text=f.read().replace("\n"," ")
text=text.lower()

#parse dataset
sent_tokenize_list = sent_tokenize(text)

token_sent = [ word_tokenize(sent) for sent in sent_tokenize_list]
token_list = []
for line in token_sent:
    token_list.append([ word for word in line if word.isalpha()])

token_list = [ ["<s>"] + line + ["</s>"] for line in token_list ]
token_sent_train ,token_sent_test = train_test_split(token_list,test_size=0.2,random_state=32)

flat_token_list_train = [item for sublist in token_sent_train for item in sublist]
flat_token_list_test = [item for sublist in token_sent_test for item in sublist]

In [107]:
# count of unigrams in corpus
unigram=Counter(flat_token_list_train)

#thus, sze of the vocabulary for given corpus is,
vocabulary_length=len(unigram)

#bigrams
bigrams=Counter()
for i in range(len(flat_token_list_train)-1):
    bigrams_tokens=(flat_token_list_train[i],flat_token_list_train[i+1])
    bigrams[bigrams_tokens]=bigrams.get(bigrams_tokens,0)+1   
        
#trigrams
trigrams=Counter()
for i in range(len(flat_token_list_train)-2):
    trigrams_tokens=(flat_token_list_train[i],flat_token_list_train[i+1],flat_token_list_train[i+2])
    trigrams[trigrams_tokens]=trigrams.get(trigrams_tokens,0)+1

#quadgrams
quadgrams=Counter()
for i in range(len(flat_token_list_train)-3):
    quadgrams_tokens=(flat_token_list_train[i],flat_token_list_train[i+1],flat_token_list_train[i+2],flat_token_list_train[i+3])
    quadgrams[quadgrams_tokens]=quadgrams.get(quadgrams_tokens,0)+1

In [109]:
#maximum likelihood estimation

#unigram
unigram_mle={}
unigram_tokens=0
for i in unigram.keys():
    unigram_tokens+=unigram[i]

for i in unigram.keys():
    unigram_mle[i]=unigram[i]/unigram_tokens

#bigrams
bigrams_mle={}
for (i,j) in bigrams.keys():
    bigrams_mle[(i,j)]=bigrams[(i,j)]/unigrams[i]

#trigrams
trigrams_mle={}
for (i,j,k) in trigrams.keys():
    trigrams_mle[(i,j,k)]=trigrams[(i,j,k)]/bigrams[(i,j)]

#quadgrams
quadgrams_mle={}
for (i,j,k,l) in quadgrams.keys():
    quadgrams_mle[(i,j,k,l)]=quadgrams[(i,j,k,l)]/trigrams[(i,j,k)]

In [126]:
print("          {0: <10} {1: <20}".format("Actual", "Possible"))
print("          {0: <10} {1: <20}".format("------", "--------"))
print("unigram   {0: <10} {1: <20}".format(vocabulary_length, vocabulary_length))
print("bigram    {0: <10} {1: <20}".format(len(bigrams), vocabulary_length**2))
print("trigram   {0: <10} {1: <20}".format(len(trigrams), vocabulary_length**3))
print("quadgram  {0: <10} {1: <20}".format(len(quadgrams), vocabulary_length**4))

          Actual     Possible            
          ------     --------            
unigram   13553      13553               
bigram    182939     183683809           
trigram   440443     2489466663377       
quadgram  589948     33739741688748481   


In [144]:
def perplexity(n,sentence):
    prob=0
    for i in range(len(sentence)-n):
        try:
            if(n==1):
                prob+=np.log2(unig_mle[sentence[i]])
            if(n==2):
                prob+=np.log2(big_mle[(sentence[i],sentence[i+1])])
            if(n==3):
                prob+=np.log2(trig_mle[(sentence[i],sentence[i+1],sentence[i+2])])
            if(n==4):
                prob+=np.log2(quad_mle[(sentence[i],sentence[i+1],sentence[i+2],sentence[i+3])])
        except:
            continue
            
    l =  prob/len(sentence)
    return np.power(2, -l)

In [155]:
def test_mean_perplexity(n, test_dataset):
    test_per=0
    for sentence in test_dataset:
        test_per+=perplexity(n,sentence)
    return test_per/len(test_dataset)

def test_perplexity(n, test_dataset):
    test_per=0
    for sentence in test_dataset:
        test_per+=perplexity(n,sentence)
    return test_per

In [160]:
print("           {0: <20} {1: <20}".format("Perplexity", "Mean Perplexity"))
print("           {0: <20} {1: <10}".format("----------", "---------------"))
print(" unigram   {0: <20} {1: <10}".format(test_perplexity(1, token_sent_test),test_mean_perplexity(1, token_sent_test)))
print(" bigrams   {0: <20} {1: <10}".format(test_perplexity(2, token_sent_test),test_mean_perplexity(2, token_sent_test)))
print(" trigrams  {0: <20} {1: <10}".format(test_perplexity(3, token_sent_test),test_mean_perplexity(3, token_sent_test)))
print(" quadgrams {0: <20} {1: <10}".format(test_perplexity(4, token_sent_test),test_mean_perplexity(4, token_sent_test)))

           Perplexity           Mean Perplexity     
           ----------           ---------------
 unigram   2632491.4177322015   375.42661405193974
 bigrams   169539.39986716444   24.178465468791277
 trigrams  18838.533056659293   2.6866133851482163
 quadgrams 8407.97183601191     1.1990832624090002


In [135]:
def findnextword(probability,next_word): #function to predict next word using multinomial distribution
    sum=0
    for item in probability:
        sum+=item
    for item in probability:
        item=item/sum
    lst=np.random.multinomial(1,probability,size=1).tolist()
    return next_word[lst[0].index(1)]

def bigramsprediction(current_word): #predicted word using bigrams
    probability=[]
    next_word=[]
    for (b1,b2) in bigrams_mle.keys():
        if (b1==current_word):
            probability.append(bigrams_mle[(b1,b2)])
            next_word.append(b2)
    return probability,next_word

def trigramsprediction(current_word,current_word1): #predicted word using trigrams
    probability=[]
    next_word=[]
    for (t1,t2,t3) in trigrams_mle.keys():
        if ((t1,t2)==(current_word,current_word1)):
            probability.append(trigrams_mle[(t1,t2,t3)])
            next_word.append(t3)
    return probability,next_word

def quadgramsprediction(current_word,current_word1,current_word2): ##predicted word using quadgrams
    probability=[]
    next_word=[]
    for (q1,q2,q3,q4) in quadgrams_mle.keys():
        if ((q1,q2,q3)==(current_word,current_word1,current_word2)):
            probability.append(quadgrams_mle[(q1,q2,q3,q4)])
            next_word.append(q4)
    return probability,next_word

In [162]:
def Generator(n): #n=2 for bigram, 3 for trigram..
    sentence_length=random.randint(10,20) #length of sentence
    sentence=["<s>"]
    length=0
    current_word,current_word1,current_word2="<s>","",""  #current words 0,1,2 for predicting next word
    
    if(n==2):
        while(length<sentence_length):
            probability,next_word = bigramsprediction(current_word)
            new_word=findnextword(probability,next_word)
            sentence.append(new_word)
            current_word=new_word
            length+=1
            if(current_word=="</s>"):
                break
    
    if(n==3):
        probability,next_word=bigramsprediction(current_word)  #predict first word using bigram
        new_word=findnextword(probability,next_word)
        sentence.append(new_word)
        current_word1=new_word
        length+=1
        
        while(length<sentence_length): #predict all other words using trigram
            probability,next_word=trigramsprediction(current_word,current_word1)
            new_word=findnextword(probability,next_word)
            sentence.append(new_word)
            current_word=current_word1
            current_word1=new_word
            length+=1
            if(current_word1=="</s>"):
                break
    if(n==4):
        probability,next_word=bigprediction(current_word) #predict first word using bigram
        new_word=findnextword(probability,next_word)
        sentence.append(new_word)
        current_word1=new_word
        
        probability,next_word=trigprediction(current_word,current_word1) #predict second word using trigram
        new_word=findnextword(probability,next_word)
        sentence.append(new_word)
        current_word2=new_word
        length+=2
        
        while(length<sentence_length): #predict all other words using quadgram
            probability,next_word=quadgramsprediction(current_word,current_word1,current_word2)
            new_word=findnextword(probability,next_word)
            sentence.append(new_word)
            current_word=current_word1
            current_word1=current_word2
            current_word2=new_word
            length+=1
            if(current_word2=="</s>"):
                break
    s=" ".join(sentence)
    print(s)

In [163]:
print("Bigram generated sentences:")        
for i in range(5):
    Generator(2) 
print("\n Trigram generated sentences:")  
for i in range(5):
    Generator(3) 
print("\n Quadgram generated sentences:")  
for i in range(5):
    Generator(4)

Bigram generated sentences:
<s> elton </s>
<s> i had lost no grammatical errors made up with a present </s>
<s> she that </s>
<s> pray what would have judged like to spend some difficulty on this is what am as isabella thorpe
<s> you i shall i should be pronounced safe and stationed quite stout girl it

 Trigram generated sentences:
<s> it would not regard it as if she was so hot and the door brought something more in
<s> you are </s>
<s> a great blessing </s>
<s> your sister loves to laugh it off as he now is i am almost ashamed
<s> mary exclaimed bless me </s>

 Quadgram generated sentences:
<s> but julia rushworth will be here presently miss woodhouse </s>
<s> lizzy said he what are you in any possible way that would
<s> you mistake me you quite mistake me she replied exerting
<s> her address to me was as much awake to the terror of a bad day amusement for you
<s> do they </s>


The sentences generated are no so readable and does not have any meaning but there seems a grammatical pattern in them 

I think neural network will perform better because in n-gram models if it sees a new word in test dataset it's probability will be zero but in neural model it might detect some pattern and assign some probability